In [1]:
pip install pysnmp

  Using cached https://files.pythonhosted.org/packages/a7/e9/d3e61ca7f167748040c8701c50572f4cc43a266327d2b43f3b3a787e02df/pysnmp-4.4.11-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6a/6e/209351ec34b7d7807342e2bb6ff8a96eef1fd5dcac13bdbadf065c2bb55c/pyasn1-0.4.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1f/fc/02361d1c2b247de73070c457c4da98c448693154894c14f2d7b48dfabf7e/pysmi-0.3.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e2/e7/4e473f5ef6fa301f4921cf9f8c3936080b2cbb4b073c2c45e5302c001097/pycryptodomex-3.9.0-cp37-cp37m-macosx_10_6_intel.whl
     |████████████████████████████████| 51kB 505kB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pysnmp.entity.rfc3413.oneliner import cmdgen
cmdGen = cmdgen.CommandGenerator()

from datetime import datetime
import os

def runtime(originalTime):
    time = originalTime // 100
    day = time // 86400
    t2 = time - (day * 86400)
    hour = t2 // 3600
    t3 = t2 - (hour * 3600)
    min = t3 // 60
    ms = time % 60
    subms = time % 100
    showTime = str(day) + ' Days, ' + str(hour) + ':' + str(min) + ':' + str(ms) + '.' + str(subms)
    return showTime

def main():
    Linux1 = ["10.10.2.19",'161',"YRUMRTG"]  # linux
    Switch1 = ["10.10.19.55",'161',"test"]    # switch
    Linux2 = ["10.10.2.20",'161',"YRUMRTG"]    # Linux
    Linux3 = ["10.10.2.21",'161',"YRUMRTG"]    # Linux
    Linux4 = ["10.10.2.200",'161',"YRUMRTG"]    # Linux
    
    serverList = [Linux1, Linux2, Linux3,Linux4, Switch1]

    uptimeName = ['System Uptime']
    uptime = ['1.3.6.1.2.1.1.3.0']

    LoadName = ['1 minute Load',
                '5 minute Load',
                '15 minute Load']
    Load = ['1.3.6.1.4.1.2021.10.1.3.1',
            '1.3.6.1.4.1.2021.10.1.3.2',
            '1.3.6.1.4.1.2021.10.1.3.3']

    CPUName = ['Percentage of user CPU time',
               'Raw user cpu time',
               'Percentages of system CPU time',
               'Raw system cpu time',
               'Percentages of idle CPU time',
               'Raw idle cpu time',
               'Raw nice cpu time']
    CPU = ['1.3.6.1.4.1.2021.11.9.0',
           '1.3.6.1.4.1.2021.11.50.0',
           '1.3.6.1.4.1.2021.11.10.0',
           '1.3.6.1.4.1.2021.11.52.0',
           '1.3.6.1.4.1.2021.11.11.0',
           '1.3.6.1.4.1.2021.11.53.0',
           '1.3.6.1.4.1.2021.11.51.0']

    memoryName = ['Total Swap Size',
                  'Available Swap Space',
                  'Total RAM in machine',
                  'Total RAM used',
                  'Total RAM Free',
                  'Total RAM Buffered',
                  'Total Cached Memory']
    memory = ['1.3.6.1.4.1.2021.4.3.0',
              '1.3.6.1.4.1.2021.4.4.0',
              '1.3.6.1.4.1.2021.4.5.0',
              '1.3.6.1.4.1.2021.4.6.0',
              '1.3.6.1.4.1.2021.4.11.0',
              '1.3.6.1.4.1.2021.4.14.0',
              '1.3.6.1.4.1.2021.4.15.0']

    oidSequenceList = [uptimeName,uptime,
                       LoadName,Load,
                       CPUName,CPU,
                       memoryName,memory]

    tmp = ""
    for server in serverList:
        SNMP_HOST = server[0]
        SNMP_PORT = server[1]
        SNMP_COMMUNITY = server[2]

        snmpData = [SNMP_HOST]
        print("="*28)
        response = os.system("ping -c 1 " + snmpData[0])
        print(SNMP_HOST,SNMP_PORT,SNMP_COMMUNITY, response)
        if response != 0:
            for i in range(0, len(oidSequenceList) - 1, 2):
                oidNameList = oidSequenceList[i]
                oidList = oidSequenceList[i + 1]
                for j in range(len(oidList)):
                    cmdGen = cmdgen.CommandGenerator()
                    errorIndication, errorStatus, errorIndex, varBinds = cmdGen.getCmd(
                        cmdgen.CommunityData(SNMP_COMMUNITY),
                        cmdgen.UdpTransportTarget((SNMP_HOST, SNMP_PORT)),
                        oidList[j])
                    if errorIndication:
                        print(errorIndication)
                        break
                    else:
                        if errorStatus:
                            print('%s at %s' % (
                                errorStatus.prettyPrint(), errorIndex and varBinds[int(errorIndex) - 1] or '?'))
                        else:
                            for name, val in varBinds:
                                if oidList[j] == '1.3.6.1.2.1.1.3.0':
                                    convert_time = str(val)
                                    showTime = runtime(int(convert_time))
                                    print(oidNameList[j], '=', showTime)
                                    snmpData.append(showTime)
                                else:
                                    print(oidNameList[j], '=', val)
                                    snmpData.append(str(val))
            tmp = str(snmpData)
            tmp = tmp.replace("', '","|")[2:-2]
            open("log.txt","a").write(tmp+"\n")

            #print(snmpData)
        else:
            for i in range(1, 19):
                snmpData.append("NA")
            open("log.txt","a").write(tmp+"\n")

            
from time import sleep 
for i in range(10):
    main()
    sleep(1)

10.10.2.19 161 YRUMRTG 0
10.10.2.20 161 YRUMRTG 0
10.10.2.21 161 YRUMRTG 0
10.10.2.200 161 YRUMRTG 512
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
10.10.19.55 161 test 0
10.10.2.19 161 YRUMRTG 0
10.10.2.20 161 YRUMRTG 0
10.10.2.21 161 YRUMRTG 0
10.10.2.200 161 YRUMRTG 512
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
10.10.19.55 161 test 0
10.10.2.19 161 YRUMRTG 0
10.10.2.20 161 YRUMRTG 0
10.10.2.21 161 YRUMRTG 0
10.10.2.200 161 YRUMRTG 512
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
No SNMP response received before timeout
10.10.19.55 161 test 0
10.10.2.19 161 YRUMRTG 0
10.10.2.20 161 YRUMRTG 0
10.10.2.21 161 YRUMRTG 0
10.10.2.200 161 YRUMRTG 512
No SNMP response received b